# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atafu village,-8.5421,-172.5159,80.96,77,72,8.08,TK,1741535225
1,1,west island,-12.1568,96.8225,84.18,89,100,8.05,CC,1741535230
2,2,morant bay,17.8815,-76.4093,83.07,71,36,10.16,JM,1741535234
3,3,port-aux-francais,-49.3500,70.2167,43.38,88,100,32.95,TF,1741535238
4,4,ushuaia,-54.8000,-68.3000,51.46,58,75,12.66,AR,1741535219


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
#%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504, 
    frame_width = 896,
    hover_cols = ["City", "Humidity"]
)

# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values - 
# max temp: 27C ~ 80.6 F
# min temp: 21 C ~ 69.8 F
# wind < 4.5
# cloudiness = 0

idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] < 80.6) & (city_data_df["Max Temp"] > 69.8)
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

2


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
111,111,afif,23.9065,42.9172,71.73,31,0,3.24,SA,1741535691
401,401,ixtapa,20.7000,-105.2000,70.83,60,0,2.30,MX,1741536942


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
111,afif,SA,23.9065,42.9172,31,
401,ixtapa,MX,20.7000,-105.2000,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
     "categories": "accommodation.hotel",
     "apiKey": geoapify_key,
     "limit" : 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"] 
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df


Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Samaya Royal Afif', 'country': 'Saudi Arabia', 'country_code': 'sa', 'state': 'Riyadh Region', 'city': 'Afif', 'street': 'شارع المعارض', 'lon': 42.8990576, 'lat': 23.883817, 'formatted': 'Samaya Royal Afif, شارع المعارض, Afif, Saudi Arabia', 'address_line1': 'Samaya Royal Afif', 'address_line2': 'شارع المعارض, Afif, Saudi Arabia', 'categories': ['accommodation', 'accommodation.hotel'], 'details': ['details'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'osm_id': 10282370809, 'name:en': 'Samaya Royal Afif', 'tourism': 'hotel', 'osm_type': 'n'}}, 'name_international': {'en': 'Samaya Royal Afif'}, 'distance': 3118, 'place_id': '51319cc6511473454059071cb8d441e23740f00103f901f986e06402000000'}, 'geometry': {'type': 'Point', 'coordinates': [42.

,City,Country,Lat,Lng,Humidity,Hotel Name
111,afif,SA,23.9065,42.9172,31,Samaya Royal Afif
401,ixtapa,MX,20.7000,-105.2000,60,Ma. Cristina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
# %%capture --no-display
# Configure the map plot
hotelPlot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504, 
    frame_width = 896,
    hover_cols = ["City", "Humidity","Hotel Name"]
)

# Display the map
hotelPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)